### Options:
* NumberOfClients = 1-23
    * *Client processes that the simulation starts*
* FractionOfClients = [0,1]  
    * *Percentage of clients perticipating each training round for fit/eval*
* NumberOfRounds = 1-*
    * *Numbers of federated rounds*
* BalancingStrategy = 'OverSampling'/'ClassWeights'/'None'
    * *Compensate for dataset class imbalance*
* DataAugmentation = 'True'/'False'
    * *To augment data sample while training, to avoid overfit*
* BatchSize = 16/32/64/128
    * *Size of each batch getting fed to the network*
* NumberOfEpochs = 1-*
    * *Number of local epochs on clients*

In [ ]:
!python3 'vitrolife_federated.py' --ExperimentName="Embryo-R10-E5-B64-ClassWeights-DA-ALL" \
                                    --NumberOfClients=23 \
                                    --FractionOfClients=1 \
                                    --NumberOfRounds=10 \
                                    --BalancingStrategy="ClassWeights" \
                                    --DataAugmentation="True" \
                                    --BatchSize=64 \
                                    --NumberOfEpochs=5

### Load final model and apply test set "federated eval"

In [ ]:
import sys
sys.path.append('../..')

import client.vitrolife.vitrolife_client as Client
import server.vitrolife.vitrolife_server as Server

list_train = []
for i in range(23):

    client = Client.VitrolifeClient(i, 5, 2, "test",  balancing_strategy="OverSampling", use_data_augmentor="False", batch_size=32)
    
    training_data = client.training_generator.__len__()
    
    #client.evaluate(None, None)
    list_train.append((training_data, f"client.-{i}"))


list_train.sort()
print(list_train)
#result = client.evaluate(None, None)

#print(result)
#model = client.model
#model.summary()

In [ ]:
import tensorflow as tf
print(tf.__version__)
#print(tf.version.VERSION)

In [ ]:

from sklearn.metrics import confusion_matrix

from numpy import save
from numpy import asarray
from numpy import load

import random as random
import time

import wandb
from wandb.keras import WandbCallback

import logging

def eval_metrics_aggregate(result):
    print("Server Last round metrics aggregation")
    fullpath = "/mnt/data/FederatedLearning/client/vitrolife/clientLogs"

    ground_truths = []
    predicted = []

    ground_truths_matrix = []
    predicted_matrix = []

    for client_id in range(23):
        current_predicted = load(f"{fullpath}/predicted_{client_id}.npy")
        current_ground = load(f"{fullpath}/ground_{client_id}.npy")
        print("Loading done")

        # Be aware of max sampling warning
        current_ground = current_ground.tolist()
        current_predicted = current_predicted.tolist()


        ground_truths_matrix.extend(current_ground)
        predicted_matrix.extend(current_predicted)

        ground_truths.extend(current_ground)
        predicted.extend(current_predicted)               

    predicted_arg_max = []
    predicted_probabilities_transformed = []

    for proba in predicted:
        proba = proba[0]
        predicted_probabilities_transformed.append([1-proba, proba])
        if proba > 0.5:
            predicted_arg_max.append(1)
        else:
            predicted_arg_max.append(0)

    predicted_arg_max = [proba.index(max(proba)) for proba in predicted_matrix]
    classes = ['No Pregnancy', 'Pregnancy']       

    wandb.sklearn.plot_confusion_matrix(ground_truths, predicted_arg_max, labels=classes)
    wandb.log({"roc" : wandb.plot.roc_curve(ground_truths, predicted_probabilities_transformed, labels=classes)})

    # wait for confustion to finsih
    time.sleep(10)
    wandb.finish()
                
        
        
experiment_name = "TestPeter"
wandb.init(project=experiment_name, entity="anton-peter")
wandb.run.name = f"Vitrolife Aggregation-Server"
eval_metrics_aggregate(None)
wandb.finish()

In [16]:
import sys
sys.path.append('../..')

import client.vitrolife.vitrolife_client as Client
import server.vitrolife.vitrolife_server as Server


sys.path.append('../..')

import utils.dataloaders.vitrolife_dataloader as DataLoader
import utils.models.vitrolife_model as Model


client_training_sizes = []
number_of_clients = 23
for client_id in range(number_of_clients):
    batch_size = 16
    balancing_strategy = "None"
    training_loader = DataLoader.VitroLifeDataloader("/mnt/data/vitroLifeDataset", batch_size , dataset_split="Train", data_augmentation = "False", balancing_strategy=balancing_strategy,clinic_ID=client_id )
    
    current_training_count = training_loader.count()
    
    client_training_sizes.append((current_training_count, client_id))


client_training_sizes.sort(reverse=True)





print([item[1] for item in client_training_sizes[0:10]])  


[7, 1, 17, 5, 2, 4, 8, 18, 12, 3]


In [ ]:
import sys

sys.path.append('../..')
import utils.dataloaders.auto_flush as AutoFlush
import utils.dataloaders.vitrolife_dataloader_old as DataLoader
import utils.models.vitrolife_model as ModelLort

import numpy as np
import tensorflow as tf

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, MaxPooling1D
from keras.layers import BatchNormalization, Reshape
from keras.models import Model

METRICS = [
    keras.metrics.TruePositives(name='tp'),
    keras.metrics.FalsePositives(name='fp'),
    keras.metrics.TrueNegatives(name='tn'),
    keras.metrics.FalseNegatives(name='fn'), 
    keras.metrics.BinaryAccuracy(name='accuracy'),
    keras.metrics.Precision(name='precision'),
    keras.metrics.Recall(name='recall'),
    keras.metrics.AUC(from_logits=True),
    keras.metrics.AUC(name='prc', curve='PR', from_logits=True), # precision-recall curve
]

model = Sequential()
model.add(Conv2D(16, (3, 3), input_shape=(250,250,1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(BatchNormalization())

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(BatchNormalization())

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(256)) # W1*x + b
model.add(Activation('relu')) # ReLU(W1*x + b)
model.add(Dropout(0.5))
model.add(Dense(1)) # W2*x + b
model.add(Activation('sigmoid'))  # softmax(W2*x + b)

# initiate RMSprop optimizer
opt = keras.optimizers.RMSprop(learning_rate=0.000001, decay=1e-6)

model.compile(loss=keras.losses.BinaryCrossentropy(from_logits=False),
              optimizer=opt,
              metrics=METRICS)
        

import resource
class MemoryCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, log={}):
        print(resource.getrusage(resource.RUSAGE_SELF).ru_maxrss)
        
batch_size = 32        
training_generator = DataLoader.VitroLifeDataloader("/mnt/data/vitroLifeDataset", batch_size , dataset_split="Train", data_augmentation ="False" , balancing_strategy="None", clinic_ID=0)
validation_generator = DataLoader.VitroLifeDataloader("/mnt/data/vitroLifeDataset", batch_size , dataset_split="Validation", clinic_ID=0)

model_gen = ModelLort.VitroLifeModel()
model_from_gen = model_gen.get_low_GPU_mem_model() 
# ...
#X_train = np.random.rand(5000, 250, 250, 1)
#Y_train = np.random.randint(0, 2, size=5000)
#X_test = np.random.rand(5000, 250, 250, 1)
#Y_test = np.random.randint(0, 2, size=5000)

list_x = []
list_y = []
for batches in range(training_generator.__len__()):
    x, y = training_generator.__getitem__(batches)
    #x, y = training_generator.__getitem__(batches)
    list_x.extend(x)
    list_y.extend(y)

list_x = np.array(list_x) 
list_y = np.array(list_y) 


model_from_gen.fit(list_x, list_y, epochs=500, callbacks=[MemoryCallback()], verbose=0)

In [ ]:
model.fit(list_x, list_y, epochs=500, callbacks=[MemoryCallback(), AutoFlush.FlushCallback()], verbose=0)

In [ ]:

import sys
sys.path.append('../..')
import client.vitrolife.vitrolife_client as Client


client = Client.VitrolifeClient(0, 10, 10, "test", "None", "false", 32)

lort = client.fit(None, None)

from numba import cuda
cuda.select_device(0)
cuda.close()

client = Client.VitrolifeClient(0, 10, 10, "test", "None", "false", 32)

lort = client.fit(None, None)

In [ ]:
import numpy
numpy.version.version

In [ ]:
pip install tensorflow-text==2.2.0

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
pip install numpy

In [8]:
import keras.backend as K
from tensorflow import keras
import numpy
import sys
sys.path.append('../..')


# 0.0000001
#model_name = "Embryo-Centralized-B16-E100-No-LrSchedule/Epoch-97"
model_name = "Embryo-Federated-B16-R100-E1-OverSampling/R-3"
model_from_disk = keras.models.load_model(f"models/{model_name}")
print(K.eval(model_from_disk.optimizer.lr))


1e-06
